In [18]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from loguru import logger

from src.text_embedder.embedder import EmbeddingsModel

from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [19]:
notebook_path = Path().resolve().parent
print(notebook_path)

# Construct the filename relative to the new path
filename = notebook_path / "test_frames_gt_new.csv"

# Read the CSV file
df = pd.read_csv(filename)
print(df)

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment
    ID                                             Prompt Length  GT_Keyframe
0    1                                  learning outcomes  short            3
1    2   What are the learning outcomes from the lecture?   long            3
2    3       What is the relation of protien and muscles?  short           26
3    4                              Essential fatty acids  short           29
4    5                                  fish oil capsules  short           30
5    6  what is the functioning of fish oil capsules a...   long           30
6    7                           calcium as micronutrient  short           46
7    8       Explain where vitamins function as coenzymes   long           41
8    9  Give a list of food items where I can attain s...   long           45
9   10          what is calcium commonly supplemented for   long           46
10  11       How are nutrients extracted from plant cells   long       

In [20]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

In [21]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [22]:
#top_3 = []
#embedding_model.check_proximity_keyframes(90, top_3)

# LOAD MODEL 

In [23]:
embedding_model = EmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [24]:
csv = notebook_path / "biology_chapter_3_3_treshhold_5.csv"
df_data_extensive = pd.read_csv(csv)
df_data_extensive

,Unnamed: 0,79,4,67,16,75,8,20,43,32,...,70,13,62,1,54,29,37,46,58,25
0,img_path,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...
1,timestamps,"[3639.0, 3639.8]","[173.08, 174.08]","[3105.48, 3240.72]","[683.96, 734.36]","[3494.72, 3509.76]","[254.04, 374.08]","[846.68, 865.04]","[1860.88, 1932.44]","[1374.08, 1477.44]",...,"[3300.36, 3309.28]","[539.04, 607.32]","[2670.0, 2761.72]","[0.0, 158.72]","[2327.76, 2387.84]","[1224.24, 1339.64]","[1601.72, 1657.0]","[2000.2, 2062.64]","[2427.88, 2462.24]","[1086.24, 1118.2]"
2,transcription,NaN,NaN,"Well, whether ingested in food or supplemente...",Remember that carbohydrates are a macromolecu...,Here are some questions which process uses tr...,We studied water in our last chapter and we l...,Foods that haven't been stripped of their nut...,Water soluble vitamins are the ones that are ...,Here's some pictures to help you visualize th...,...,Facilitated diffusion is another way that sub...,Many people might choose to rehydrate themsel...,The mitochondria are an organelle that use gl...,"Welcome to Chapter 3, Nutrients, Membrings an...","As we learned in our last chapter, some membr...","Most mammals, including us humans, store fat ...","Unfortunately, this process creates something...",Calcium is a commonly supplemented mineral. Y...,"As you can see in the picture here, these pro...",Complete proteins contain all the essential a...
3,ocr_extracted_text,Transport Across Membranes\r\n\r\n+ Exocytosis...,Do you use any of these supplements?\r\n\r\n>\...,Transport Across Membranes\r\n\r\n* Membrane t...,Nutrients: Macronutrients\r\n\r\n* Carbohydrat...,Which process uses transport proteins to move\...,Nutrients: Macronutrients\r\n\r\n» Water\r\n— ...,Nutrients: Macronutrients\r\n\r\n* Whole foods...,Nutrients: Micronutrients\r\n\r\n+ Water-solub...,NaN,...,Transport Across Membranes\r\n\r\n* Passive tr...,Nutrients: Macronutrients\r\n\r\n* Sports drin...,Cell Structure\r\n\r\n+ Subcellular structures...,Chapter 3:\r\n\r\nNutrients & Membrane Transpo...,Cell Structure\r\n\r\n* Membrane structure\r\n...,Nutrients: Macronutrients\r\n\r\n* Fats: sourc...,Nutrients: Macronutrients\r\n\r\n+ Fats\r\n— H...,Nutrients: Micronutrients\r\n\r\n* Calcium\r\n...,Cell Structure\r\n\r\n* Membrane structure\r\n...,Nutrients: Macronutrients\r\n\r\n+ Proteins: p...
4,llava_result,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The image you've provided appears to be a sli...,...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appe

In [25]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)
llm_long_summary = llm_long_summary[1:]
logger.info(llm_long_summary)

2024-07-13 13:09:02.410 | INFO     | __main__:<module>:7 - ['Transport Across Membranes: The slide explains the three main types of transport across cell membranes: simple diffusion, facilitated diffusion, and active transport. Simple diffusion involves the movement of substances from an area of higher concentration to an area of lower concentration without energy input. Facilitated diffusion uses carriers to move substances against their concentration gradient, while active transport requires energy provided by ATP to move substances against their concentration gradient. The slide also notes exocytosis, a type of cellular secretion where substances are released from the cell membrane into the extracellular space through the fusion of vesicles with the membrane and expulsion of large molecules outside the cell.', 'The slide appears to be discussing common dietary supplements and their potential benefits as well as side effects. The lecture likely intends to engage students or audience 

In [26]:
# get ocr_only
# Since ocr_only embeddings can't be mapped to floats due to e-02, e-01, we're generating new embeddings 
ocr_only = df_data_extensive.iloc[3].to_dict().values()

# convert to list
ocr_only = list(ocr_only)

ocr_only = ocr_only[1:]

for i in ocr_only:
    logger.info(i)

2024-07-13 13:09:02.906 | INFO     | __main__:<module>:11 - Transport Across Membranes

+ Exocytosis: a membrane-bound vesicle fuses with the
membrane and expels the large molecule outside the cell

(2) Exocytosis


2024-07-13 13:09:02.916 | INFO     | __main__:<module>:11 - Do you use any of these supplements?

>

gingko
kava
ginseng

melatonin

mooo

coconut water

‘$0 YOU WAVE VITAMINS, AND THEN YOU HAVE "HERBAL
SUPPLEMENTS." WHICH ARE WOT FDA APPROVED.

L
‘AND JUST FOR REFERENCE, THE TWINKIE IS.


2024-07-13 13:09:02.917 | INFO     | __main__:<module>:11 - Transport Across Membranes

* Membrane transport
— Plasma membrane:
phospholipid bilayer is
differentially permeable
= Hydrophobic substances


2024-07-13 13:09:02.918 | INFO     | __main__:<module>:11 - Nutrients: Macronutrients

* Carbohydrates: main energy

source Fructose e
+ Possible structural composition: =
— Single-unit simple sugars
(monosaccharides)
~ Many sugar subunits o
(polysaccharides) (s)crapefined 2) Grapes
uy

*

In [27]:
# get ocr_lava 
lava_only = df_data_extensive.iloc[4].to_dict().values()

lava_only = list(lava_only)

lava_only = lava_only[1:]

concat_result_ocr_lava = []

# list to a list of strings
ocr_only = [str(i) for i in ocr_only]

lava_only = [str(i) for i in lava_only]

for i in range(len(ocr_only)):
    concat_result_ocr_lava.append(ocr_only[i] + lava_only[i])

In [28]:
# get ocr_transcriptions
transcriptions = df_data_extensive.iloc[2].to_dict().values()

transcriptions = list(transcriptions)
transcriptions = transcriptions[1:]

concat_result_ocr_transcriptions = []

# concatenate the result 
ocr = ocr_only

transcriptions = [str(i) for i in transcriptions]
ocr_only = [str(i) for i in ocr_only]

# for i in transcriptions:
#     if transcriptions[i] == 'nan' or transcriptions[i] == None:
#         transcriptions[i] = ''

for i in range(len(ocr_only)):
    concat_result_ocr_transcriptions.append(ocr_only[i] + transcriptions[i])

logger.info(concat_result_ocr_transcriptions)

2024-07-13 13:09:03.802 | INFO     | __main__:<module>:22 - ['Transport Across Membranes\r\n\r\n+ Exocytosis: a membrane-bound vesicle fuses with the\r\nmembrane and expels the large molecule outside the cell\r\n\r\n(2) Exocytosis\r\n\r\nnan', 'Do you use any of these supplements?\r\n\r\n>\r\n\r\ngingko\r\nkava\r\nginseng\r\n\r\nmelatonin\r\n\r\nmooo\r\n\r\ncoconut water\r\n\r\n‘$0 YOU WAVE VITAMINS, AND THEN YOU HAVE "HERBAL\r\nSUPPLEMENTS." WHICH ARE WOT FDA APPROVED.\r\n\r\nL\r\n‘AND JUST FOR REFERENCE, THE TWINKIE IS.\r\n\r\nnan', "Transport Across Membranes\r\n\r\n* Membrane transport\r\n— Plasma membrane:\r\nphospholipid bilayer is\r\ndifferentially permeable\r\n= Hydrophobic substances\r\n\r\n Well, whether ingested in food or supplemented in pills, one substances reach cells,  to be used they must go across the cell membrane.  Molecules must cross that plasma membrane to gain access to the inside of the cell  where they can be used by those cell structures and organelles to syn

In [29]:
# get img_path
img_path = df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path = img_path[1:]

In [30]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary.csv', index=False)


2024-07-13 13:09:04.290 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
2024-07-13 13:09:15.212 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[ 0.0569, -0.0283, -0.0086,  ...,  0.3632,  0.3583,  0.0850],
        [-0.2071, -0.1645, -0.0042,  ...,  0.1312, -0.0608,  0.2009],
        [-0.0083,  0.1942, -0.0935,  ...,  0.1645,  0.2800, -0.1498],
        ...,
        [-0.1003, -0.2544,  0.4474,  ..., -0.2105,  0.2100, -0.1704],
        [-0.1289, -0.0912, -0.1340,  ...,  0.0765,  0.2274, -0.0715],
        [-0.2274, -0.1598,  0.0243,  ..., -0.0246,  0.0617, -0.0040]])
2024-07-13 13:09:15.212 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-13 13:09:15.252 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.5148, 0.5050, 0.3735]),
indices=tensor([62, 63,  1])) - GT is keyframe number 3
2024-07-13 13:09:15.261 | INFO     | src.text_embedder.embe

<Figure size 800x2000 with 0 Axes>

In [31]:
# Embedd with OCR ONLY 
logger.info(f"Embedd with OCR ONLY ")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

# need to be embedded

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_only)

embedding_model.text_embeddings = embeddings

# already embedded 
#
# if isinstance(embedding_model.text_embeddings, list):
#     for i, text_embedding in enumerate(embedding_model.text_embeddings):
#         logger.info(text_embedding)
#         # Regular expression pattern to find numeric values
#         numeric_pattern = r"[-+]?\d*\.\d+(?:[eE][-+]?\d+)?"
#         
#         # Extract numeric values from the tensor string
#         tensor_values_str = re.findall(numeric_pattern, text_embedding)
#         
#         # Convert the extracted string values to float
#         tensor_values = [float(value) for value in tensor_values_str]
#         
#         # Create a PyTorch tensor from the list of floats
#         embedding_model.text_embeddings[i] = torch.tensor(tensor_values)
#         
# 
# # create one single torch for sim search 
# embedding_model.text_embeddings = torch.stack(embedding_model.text_embeddings, dim=0)
# 
# logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_standard_ocr_only.csv', index=False)
logger.info('Saved old_df_standard_ocr_only.csv')

2024-07-13 13:09:17.197 | INFO     | __main__:<module>:2 - Embedd with OCR ONLY 
2024-07-13 13:09:27.431 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-13 13:09:27.477 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3216, 0.2852, 0.2718]),
indices=tensor([ 8, 63, 62])) - GT is keyframe number 3
2024-07-13 13:09:27.477 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:159 - Length of img paths: 85
2024-07-13 13:09:27.477 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - #####GT is keyframe number 3#####
2024-07-13 13:09:27.477 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Max similarity for index 8 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-032-01.jpg
2024-07-13 13:09:27.477 | INFO     | src.text_

<Figure size 800x2000 with 0 Axes>

In [32]:
# Embedd with OCR TRANSCRIPTIONS
# TODO: Doesn't work since transcriptions are nans
logger.info(f"Embedd with TRANSCRIPTIONS")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(concat_result_ocr_transcriptions)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_transcriptions.to_csv('df_standard_df_ocr_transcriptions.csv', index=False)

2024-07-13 13:09:28.953 | INFO     | __main__:<module>:3 - Embedd with TRANSCRIPTIONS
2024-07-13 13:09:39.820 | INFO     | __main__:<module>:19 - Embedding Model.text_embeddings: tensor([[-0.0563,  0.0627, -0.0358,  ...,  0.1612,  0.4490, -0.2317],
        [-0.2135, -0.2632,  0.2310,  ..., -0.0914,  0.1228, -0.0360],
        [-0.0592,  0.2447, -0.0615,  ...,  0.0526,  0.2782, -0.0641],
        ...,
        [-0.1420, -0.2381,  0.3573,  ..., -0.2036,  0.2723, -0.1992],
        [-0.1356, -0.0717, -0.0947,  ...,  0.1162,  0.3729, -0.2062],
        [ 0.0950, -0.1783,  0.0910,  ..., -0.1368,  0.1797, -0.1441]])
2024-07-13 13:09:39.820 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-13 13:09:39.851 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3129, 0.2720, 0.2055]),
indices=tensor([63, 62, 46])) - GT is keyframe number 3
2024-07-13 13:09:39.851 | INFO     | src.text_embedder.embedd

<Figure size 800x2000 with 0 Axes>

In [33]:
# Embedd with OCR LAVA
logger.info(f"Embedd with OCR LAVA")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(concat_result_ocr_lava)

embedding_model.text_embeddings = embeddings

rows = []

rows = get_results(df)

df_ocr_lava = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_lava.to_csv('df_standard_ocr_lava.csv', index=False)

2024-07-13 13:09:41.212 | INFO     | __main__:<module>:2 - Embedd with OCR LAVA
2024-07-13 13:09:49.719 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-13 13:09:49.750 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:157 - Top 3 Similarity scores: torch.return_types.topk(
values=tensor([0.3036, 0.2686, 0.2421]),
indices=tensor([63, 62, 16])) - GT is keyframe number 3
2024-07-13 13:09:49.758 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:159 - Length of img paths: 85
2024-07-13 13:09:49.758 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - #####GT is keyframe number 3#####
2024-07-13 13:09:49.758 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Max similarity for index 63 is the keyframe /Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-003-01.jpg
2024-07-13 13:09:49.758 | INFO     | src.text_

<Figure size 800x2000 with 0 Axes>